
In this section, we will be using active learning to predict. 

We will be using a dataset that is about product marketing - bank product marketing to be more precise. We want to predict if the client will subscribe to a term deposit. The situation is that customer agents, whose time costs money, will call customers. We need to decide which customers will be most likely to sign up, so we can prioritze the calls. 

This is a nice use case for active learning, because - not unlike in reinforcement learning - uncertainty could be an additional criterion, apart from positive expectation, from a customer. Over time, this entropy-reduction seeking behavior would reduce as the model's understanding of customers improves.

Again we will get the data from OpenML.

In [1]:
pip install -q openml modAL category_encoders

Note: you may need to restart the kernel to use updated packages.


In [2]:
import openml
dataset = openml.datasets.get_dataset(1461)

In [3]:
X, y, categorical_indicator, _ = dataset.get_data(
    dataset_format='dataframe',
    target=dataset.default_target_attribute
)
categorical_features = X.columns[categorical_indicator]
numeric_features = X.columns[[not(i) for i in categorical_indicator]]

In [4]:
X.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16
0,58.0,management,married,tertiary,no,2143.0,yes,no,unknown,5.0,may,261.0,1.0,-1.0,0.0,unknown
1,44.0,technician,single,secondary,no,29.0,yes,no,unknown,5.0,may,151.0,1.0,-1.0,0.0,unknown
2,33.0,entrepreneur,married,secondary,no,2.0,yes,yes,unknown,5.0,may,76.0,1.0,-1.0,0.0,unknown
3,47.0,blue-collar,married,unknown,no,1506.0,yes,no,unknown,5.0,may,92.0,1.0,-1.0,0.0,unknown
4,33.0,unknown,single,unknown,no,1.0,no,no,unknown,5.0,may,198.0,1.0,-1.0,0.0,unknown


In [5]:
from matplotlib import pyplot as plt
import pandas as pd
%matplotlib inline
import seaborn as sns

In [6]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
import category_encoders as ce


def col_names2slices(column_names, X):
    numpy_slice = []
    X_cols = list(X.columns)
    for col in column_names:
        numpy_slice.append(X_cols.index(col))
    return list(numpy_slice)

ordinal_encoder = ce.OrdinalEncoder(
    cols=None,  # all features that it encounters
    handle_missing='return_nan',
    handle_unknown='ignore'
).fit(X)

preprocessor = ColumnTransformer(
    transformers=[
        #('ranges', range_transformer, range_features),
        ('cat', ordinal_encoder, categorical_features),
        ('num', FunctionTransformer(validate=False), numeric_features)]
)

preprocessor = preprocessor.fit(X)

/Users/ben/anaconda3/lib/python3.6/site-packages/category_encoders/utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


Since we are dealing with an imbalanced dataset, let us use class weights. This basically means that we are upsampling the minority (signing up) class and downsampling the majority class (not signing up). The formula for the class weights is as follows:

$ \text{n_samples} / (\text{n_classes} * \text{np.bincount}(y))$

In [7]:
class_weights = len(X) / (y.astype(int).value_counts() * 2)

Only few models in [sklearn allow incremental or online learning](https://scikit-learn.org/0.15/modules/scaling_strategies.html). A few linear models include the *partial_fit()* method. The Sk-multiflow package specializes in incremental and online/streaming models. Let us try one of their models.

In [8]:
pip install -q scikit-multiflow

Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
from skmultiflow.trees import HoeffdingTree
from sklearn.metrics import roc_auc_score
from modAL.uncertainty import classifier_uncertainty
import random


class ActivePipeline:
    '''inspired by modAL.models.ActiveLearner
    '''
    def __init__(self, model, preprocessor, class_weights):
        self.model = model
        self.preprocessor = preprocessor
        self.class_weights = class_weights

    @staticmethod
    def values(X):
        if isinstance(X, (np.ndarray, np.int64)):
            return X
        else:
            return X.values

    def preprocess(self, X):
        X_ = pd.DataFrame(
            data=self.values(X),
            columns=['V1', 'V2', 'V3', 'V4',
                     'V5', 'V6', 'V7', 'V8',
                     'V9', 'V10', 'V11',
                     'V12', 'V13', 'V14',
                     'V15', 'V16']
        )
        return self.preprocessor.transform(X_)

    def fit(self, X, ys):
        weights = [self.class_weights[y] for y in ys]
        self.model.fit(self.preprocess(X), self.values(ys))  #, classes=ys.unique(), weight=weights)  # classes

    def update(self, X, ys):
        if isinstance(ys, (int, float)):
            weight = self.class_weights[y]
        else:
            weight = [self.class_weights[y] for y in ys]
        self.model.partial_fit(
            self.preprocess(X),
            self.values(ys),
            weight
        )

    def predict(self, X):
        return self.model.predict(
            self.preprocess(X)
        )

    def predict_proba(self, X):
        return self.model.predict_proba(
            self.preprocess(X)
        )

    def max_margin_uncertainty(self, X):
        '''similar to modAL.uncertainty.margin_uncertainty

        Returns
        - the index of the highest margin uncertainty
        - the uncertainty
        '''
        probs = self.predict_proba(X)
        uncertainties = ((probs[:,2] - probs[:, 1]) + 1.0) / 2.0
        ind = random.choices(range(len(uncertainties)), uncertainties)
        #ind = np.argmax(uncertainties)
        return ind, uncertainties[ind]

    def score(self, X, y):
        return roc_auc_score(y, self.predict(X))

active_pipeline = ActivePipeline(HoeffdingTree(), preprocessor, class_weights.to_dict())
active_pipeline.model.classes = [0, 1, 2]

/Users/ben/anaconda3/lib/python3.6/site-packages/skmultiflow/trees/hoeffding_tree.py:35: FutureWarning: 'HoeffdingTree' has been renamed to 'HoeffdingTreeClassifier' in v0.5.0.
The old name will be removed in v0.7.0
  "The old name will be removed in v0.7.0", category=FutureWarning)


In [10]:
from collections import namedtuple

X_all = X.copy().reset_index(drop=True)
y_all = y.copy().reset_index(drop=True)

N = len(X)
n_initial = 100
initial_idx = np.random.choice(
    len(X_all), size=n_initial, replace=False
)
X_train, y_train = X_all.iloc[initial_idx], y_all.iloc[initial_idx]

active_pipeline.fit(X_train, y_train.astype(int))

In [ ]:
Iteration = namedtuple(
    'Iteration',
    ['iteration',
     'score',
     'uncertainty',
     'target']
)

history = []
it = 0
learned = []
score = active_pipeline.score(X, y)
print(score)
while score < 0.90 and it <= N:
    stream_idx, uncertainty = active_pipeline.max_margin_uncertainty(X_all)
    active_pipeline.update(
        np.array(X_all.iloc[stream_idx], ndmin=2),
        np.array(y_all.astype(int).iloc[stream_idx], ndmin=1)
    )
    score = active_pipeline.score(X, y)
    history.append(
        Iteration(
            iteration=it,
            score=score,
            uncertainty=uncertainty,
            target=y_all.iloc[stream_idx]
        )
    )
    X_all = X_all.drop(stream_idx).reset_index(drop=True)
    y_all = y_all.drop(stream_idx).reset_index(drop=True)
    it += 1


In [ ]:
history[1].iteration

In [ ]:
len(history)

In [ ]:
plt.plot(
    [hist.iteration for hist in history if not isinstance(hist, float)],
    [hist.uncertainty for hist in history if not isinstance(hist, float)],
    color='red',
    marker='o',
    linestyle='dashed',
    linewidth=1,
    markersize=8
)
plt.plot(
    [hist.iteration for hist in history if not isinstance(hist, float)],
    [hist.score for hist in history if not isinstance(hist, float)],
    color='blue',
    marker='o',
    linestyle='dashed',
    linewidth=1,
    markersize=8
)

plt.xlabel("iteration")
#plt.ylabel("uncertainty")
plt.legend(["uncertainty", 'score'])


In [ ]:
[hist.score for hist in history if not isinstance(hist, float)],

In [ ]:
from mlxtend.plotting import plot_decision_regions
plot_decision_regions(X_train, y_train, clf=resampling_pipeline_brf, legend=2)